In [18]:
import os
import numpy as np
from datetime import datetime

import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.datasets.factory import get_imdb
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)



# hyper-parameters
# ------------
imdb_name = 'sunrgbd_test_rgb_i_100_8bits'train
cfg_file = 'experiments/cfgs/faster_rcnn_end2end_sunrgbd.yml'
pretrained_model = 'data/pretrained_model/VGG_imagenet.npy'
output_dir = 'models/sunrgbd_rgb100/'




# load config
cfg_from_file(cfg_file)

# load data
imdb = get_imdb(imdb_name)

<bound method sunrgbd.default_roidb of <faster_rcnn.datasets.sunrgbd.sunrgbd object at 0x7ff43063e790>>


In [19]:
gt_roidb=imdb.gt_roidb()

wrote gt roidb to /c16/THESE.JORIS/datasets/SUNRGBD_pv/faster_rcnn/cache/sunrgbd_test_rgb_i_100_8bits_gt_roidb.pkl


In [20]:
# print "analysing..."
myClassesDict = {}
for img_rois in gt_roidb:
    for id_cls in img_rois['gt_classes']:
        myClassesDict[imdb._classes[id_cls]] = myClassesDict.get(imdb._classes[id_cls], 0) + 1
        
a = sorted(myClassesDict, key=myClassesDict.get)
for cls in a:
    print cls, myClassesDict[cls]
# print myClassesDict
# print "analysed !"

shower_curtain 1
blinds 66
bathtub 80
window 84
towel 143
bag 145
books 181
toilet 186
fridge 293
person 320
mirror 337
curtain 348
picture 388
tv 497
whiteboard 652
sink 695
paper 757
night_stand 817
dresser 881
bed 949
counter 1468
bookshelf 1499
lamp 2590
box 2648
cabinet 2703
sofa 2824
door 4441
table 10686
pillow 11720
desk 13342
chair 126195
